<a href="https://colab.research.google.com/github/galeone/italian-machine-learning-course/blob/master/Object_Detection_and_Classification_%2B_TensorFlow_Hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x 
import tensorflow as tf



## Object detection and classification + Transfer Learning

Definire una rete neurale convouzionale con "due teste" (double headed convolutional neural network) in modo tale da risolvere il problema di classificazione e localizzazione di un oggetto all'interno dell'immagine.

È necessario definire la rete con due distinte "teste" in quanto:

- la regression head, avrà l'unico scopo di regredire le coordinate dell'oggetto presente all'interno dell'immagine
- la classification head, classificherà l'immagine.

Per poter implementare questo tipo di rete, siamo vincolati ad usare la **functional** API di Keras, in quanto la topologia non è un semplice stack di layer, ma si biforca nella parte finale.

L'architettura è formata da:

- una parte di feature extraction, che altro non è che uno stack di layer convoluzionali, incaricati di estrarre feature utili per i layer di regressione e classificazione
- le due teste che, usando le stesse feature estratte dal feature extractor, apprenderanno la capacità di classificare e regredire le coordinate della boundingbox (rispettivamente).

Per implmenetare il feature extractor abbiamo due opzioni:

1. Scrivere il feature extractor e ri-allenarlo da zero
2. Utilizzare un modello pre-trainato

Li vedremo entrambi.

## Preparazione del dataset (subset PASCAL VOC 2012)

Esistono diversi dataset di object detection; un'immagine contiene 1+ oggetti annotati, con annotazione delle 4 coordinate + la classe.

Dato che siamo interessati a risolvere il problema dell'object localization + classifcation, dobbiamo estrarre un sottoinsieme delle immagini di un dataset di object detection, cercando solo le immagini che contengono una (ed una sola) bounding box per immagine.

Il dataset che andremo ad utilizzare è il dataset **PASCAL VOC 2012**: questo dataset è un dataset usato per fare benchmark di algoritmi di image classifcation, object detection, semantic segmentation e human pose estimation.


In [0]:
!pip install --upgrade tensorflow_datasets

In [0]:
import tensorflow_datasets as tfds

In [0]:
# Get the dataset (splits are OK)
# Train, test, and validation are datasets for object detection: multiple objects per image. 
(train, test, validation), info = tfds.load(
    "voc2007", split=["train", "test", "validation"], with_info=True
)

Ottenuto il dataset, dobbiamo scrivere:

1. Una funzione che disegni sull'immagine la bouning box dell'oggetto trovato
2. La funzione che filtri il dataset ed estragga solo le immagini con una singola annotazione

Per poter capire come lavorare sul dataset, sfruttiamo l'oggetto `info`.

In [0]:
print(info)

È facile notare che per ogni immagine è presenta una `Sequence` di oggetti, con varie informazioni a disposizioni, tra le quali le 4 coordinate (assolute, valori tra [0,1]), la label, ed altre informazioni sull'oggetto.

Per avere un'idea del contenuto dataset, utilizziamo la funzione `tfds.show_examples` e dopo definitiamo la funzione filtro che utilizzeremo per estrarre il sottoinsieme da noi desiedrato per ogni split.

In [0]:
fig = tfds.show_examples(info, train, rows=4, cols=4)

Differentemente dai dataset per la classificazione, qui non vediamo le bounding boxes disegnate attorni agli oggetti ne tantomeno la label associata all'immagine (perché solitamente ce ne è più di una).

Possiamo passare ora alla definizione della funzione filtro ed alla sua applicazione agli split, seguita dalla definizione della funzione di disegno **di una singola bounding box** sull'immagine.

In [0]:
# Create a subset of the dataset by filtering the elements: we are interested 
# in creating a dataset for object detetion and classification that is a dataset 
# of images with a single object annotated. 
def filter(dataset): 
    return dataset.filter(lambda row: tf.equal(tf.shape(row["objects"]["label"])[0], 1)) 
 
train, test, validation = filter(train), filter(test), filter(validation)

## Creazione della pipeline di input

Il dataset filtrato è pronto, dobbiamo solo costrire la pipeline di input utilizzando il method chaining e costruire i batch.

Dato che le immagini sono tutte di dimensione differente, dobbiamo scalarle ad una risoluzione uguale in modo da poter creare dei batch.

La risoluzione scelta è `(299,299)` - la scelta della risoluzione non è casuale, ma è quella che si aspetta il feature extractor pre-trainato che utilizzeremo in seguito (**inception-v3**).

In [0]:
def prepare(row):
  # Scaling in [0,1] because inception v3 expects input values in this range
  row["image"] = tf.image.convert_image_dtype(row["image"], tf.float32) 
  row["image"] = tf.image.resize(row["image"], (299, 299))
  return row

train = train.map(prepare).batch(32).prefetch(1)
validation = validation.map(prepare).batch(32).prefetch(1)
test = test.map(prepare).batch(32).prefetch(1)

Possiamo anche definire ed utilizzare un funzione di comodo, che date le immagini e le bounding box corrispondenti, le disegni.

L'API di TensorFlow offre già la funzione `tf.image.draw_bounding_boxes` da utilizare, noi ci limitiamo ad utilizzarla gestendo,però, i possibili casi di input.

In [0]:
def draw(images, boxes, color=tf.constant((1.0, 1.0, 0, 0))):
  images = tf.image.convert_image_dtype(images, tf.float32)
  if tf.equal(tf.rank(images), 3):
    print("expanded dims")
    images = tf.expand_dims(images, axis=[0])
  if tf.equal(tf.rank(boxes), 2):
    boxes = tf.expand_dims(boxes, axis=[1])
  # draw bounding boxes wants [batch_size, num boxes, coordinates]
  images = tf.image.draw_bounding_boxes(
      images=images,
      boxes=boxes,
      colors=tf.reshape(color, (1, 4)))
  return images

Vista l'integrazione con matplotlib dei jupyter notebook, possiamo provare a visualizzare direttamente qui qualche immagine con la relativa bounding box.

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

# take only a batch and loop over it
for row in train.take(1):
  images = draw(row["image"], row["objects"]["bbox"])
  for idx, image in enumerate(images):
    tf.print("label: ", info.features["objects"]["label"].int2str(row["objects"]["label"][idx][0])) 
    plt.imshow(image) 
    plt.show()

## Feature-extractor from scratch

Definire l'architettura bifronte è facile usando le keras API.

Una particolarità dei Keras model, è la loro possibilità di essere utilizzati come layer all'interno di altri modelli più complessi.
Possiamo quindi definire una rete formata da tre blocchi:

- feature extractor: trasforma l'immagine in in layer di feature a bassa dimensionalità
- classification head: un classificatore in grado di classificare l'input nelle n classi supportate
- regression head: possiamo definirlo in due modi diversi; o class agnostic (agnostico alla classe) e quidi un regressore di 4 coordinate; oppure class specific, in cui la testa è formata da `4*<numero classi>` neuroni di output.

Definire la regression head class agnostic è più facile (ed è efficace quasi in egual modo), quindi definiremo la testa in questo modo.

In [0]:
# Feature extractor layer
inputs = tf.keras.layers.Input(shape=(299,299,3))
net = tf.keras.layers.Conv2D(32, (3,3),strides=(2,2),padding='same',activation=tf.nn.relu)(inputs)
net = tf.keras.layers.Conv2D(64, (3,3),strides=(2,2),padding='same',activation=tf.nn.relu)(net)
net = tf.keras.layers.Conv2D(128, (3,3),strides=(2,2),padding='same',activation=tf.nn.relu)(net)
net = tf.keras.layers.Conv2D(256, (3,3),strides=(2,2),padding='same',activation=tf.nn.relu)(net)
net = tf.keras.layers.Conv2D(512, (3,3),strides=(2,2),padding='same',activation=tf.nn.relu)(net)
net = tf.keras.layers.Conv2D(1024, (3,3),strides=(2,2),padding='same',activation=tf.nn.relu)(net)
features = tf.keras.layers.Flatten()(net)

# Regression head
net = tf.keras.layers.Dense(512)(features)
net = tf.keras.layers.ReLU()(net)
coordinates = tf.keras.layers.Dense(4, use_bias=False)(net)

# Classification head
net = tf.keras.layers.Dense(1024, activation=tf.nn.relu)(features)
net = tf.keras.layers.Dense(512, activation=tf.nn.relu)(net)
classes = tf.keras.layers.Dense(20)(net)
chimera = tf.keras.Model(inputs=inputs, outputs=[classes, coordinates])

## Loss function e training loop

Allenare in parallelelo una rete neurale per due task differenti è il compito del **multi-task learning**.

Nella pratica, trattasi di definire una loss function che tenga in considerazione entrambi i problemi da risolvere.

In questi casi, la loss diventa una loss pesata formata da due termini:

- un termine di classificazione
- un termine di regressione delle coordinate

$$ \mathcal{L} = \lambda_1 \mathcal{L}_{c} + \lambda_2 \mathcal{L}_{r} $$

In [0]:
def l2(y_true, y_pred):
  return tf.reduce_mean(tf.reduce_sum(tf.square(tf.squeeze(y_true, axis=[1]) - y_pred), axis=[1]))

regression_loss = l2
classification_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
lambda_1 = 1
lambda_2 = 1

def loss(y_true, y_pred, box_true, box_pred):
  return lambda_1 * classification_loss(y_true, y_pred) + lambda_2 * regression_loss(box_true, box_pred)

# Define the optimizer
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)

# common name
model = chimera

# Metrics (TOOD)
mean_loss = tf.keras.metrics.Mean(name="loss")

# writers
train_writer = tf.summary.create_file_writer("odlogs/train")
validation_writer = tf.summary.create_file_writer("odlogs/validation")
test_writer = tf.summary.create_file_writer("odlogs/test")

def compute_loss(input_samples):
    predictions, coordinates = chimera(input_samples["image"])
    loss_value = loss(
        input_samples["objects"]["label"], predictions,
        input_samples["objects"]["bbox"], coordinates)
    return loss_value

@tf.function
def train_step(input_samples):
  with tf.GradientTape() as tape:
    loss_value = compute_loss(input_samples)

  gradient = tape.gradient(loss_value, model.trainable_variables)
  optimizer.apply_gradients(zip(gradient, model.trainable_variables))
  return loss_value

def measure_metrics(input_samples):
  # TODO: accuracy metric on bale
  # TODO: IoU over predictions
  mean_loss.update_state(compute_loss(input_samples))

## Logging: TensorBoard

Dato che TensorBoard è lo strumento offerto da TensorFLow per il logging delle metriche e dei dati, lo utilizziamo al posto dei notebook per la data visualization durante la fase di training.

In [0]:

%load_ext tensorboard
%tensorboard --logdir odlogs/

## Definizione del training loop

In [0]:
global_step = tf.Variable(0, dtype=tf.int64, trainable=False)
epoch_counter = tf.Variable(0, dtype=tf.int64, trainable=False)

def train_loop(num_epochs):
  
  for epoch in tf.range(epoch_counter, num_epochs):
    for input_samples in train:
      loss_value = train_step(input_samples)
      measure_metrics(input_samples)
      global_step.assign_add(1)

      if tf.equal(tf.math.mod(global_step, 10), 0):
        mean_loss_value = mean_loss.result() 
        # TODO metrics
        mean_loss.reset_states()
        tf.print(f"[{global_step.numpy()}] loss value: ", mean_loss_value)
        with train_writer.as_default():
          tf.summary.scalar("loss", mean_loss_value, step=global_step)
          # Draw ground truth (yellow)
          logme =  draw(input_samples["image"],
                                input_samples["objects"]["bbox"],
                                color=tf.constant((1.0, 0, 0, 0)))
          # Draw prediction (red)
          predicted_classes, predicted_boxes = model(input_samples["image"])
          logme = draw(logme, predicted_boxes)
          tf.summary.image("gt_vs_prediction", logme, step=global_step, max_outputs=5)
    # end of epoch: measure performance on validation set and log the values on tensorboard
    tf.print(f"Epoch {epoch.numpy() + 1 } completed")
    epoch_counter.assign(epoch + 1)
    # TODO: insert validation code here

In [0]:
train_loop(5)

## Pretrained feature extractor: TensorFlow Hub

TensorFlow Hub è un "hub" in cui è possibile trovare modelli pre-allenati printi all'uso. I modelli presenti sono perfettamente integrati con la Keras API e TensorFlow 2.0.

La lista completa dei modelli è disponibile sul sito uffuciale: https://tfhub.dev/

Essendo l'ecosistema TensorFlow modulare, TensorFlow Hub è un pacchetto Python installabile singolarmente.

In [0]:
! pip install --upgrade tensorflow_hub

L'integrazione con Keras è straordinaria: un  solo metodo permette di **scaricare**, **salvare**, utilizzare sia come feature extractor fisso che ri-allenare il modello.

Il modello scelto è **Inception V3**.

![inc](https://i.imgur.com/HpFoGP0.png)

In [0]:
import tensorflow_hub as hub
import os

os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "1"

# Feature extractor layer
inputs = tf.keras.layers.Input(shape=(299,299,3))
features = hub.KerasLayer(
        "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/2",
        output_shape=[2048],
        trainable=False,
      )(inputs)

# Regression head
net = tf.keras.layers.Dense(512)(features)
net = tf.keras.layers.ReLU()(net)
coordinates = tf.keras.layers.Dense(4, use_bias=False)(net)

# Classification head
net = tf.keras.layers.Dense(1024, activation=tf.nn.relu)(features)
net = tf.keras.layers.Dense(512, activation=tf.nn.relu)(net)
classes = tf.keras.layers.Dense(20)(net)
chimera = tf.keras.Model(inputs=inputs, outputs=[classes, coordinates])

## Esercizio 1

Misurare la classification accuracy nel training loop "feature extraction from scratch".

## Esercizio 2

Misurare le performance di validation al termine di ogni epoca nel training loop "feature extraction from scratch".

## Esercizio 3

Implementare il salvataggio ed il restore dello stato del training loop, mediante i checkpoint.

## Esercizio 4

Implementare il training loop completo per il modello definito usando il pre-trained feature extractor.

## Esercizio 5

Provare ad allenare il modello che usa Inception v3 sia facendo transfer learning che facendo fine tuning.
Misurare la diferenza di tempo di esecuzione mediante usando `from time import time()`

## Esercizio 6

Data la funzione per il calcolo dell'intersection over union:

```python

def iou(pred_box, gt_box, h, w):
    """
    Compute IoU between detect box and gt boxes
    Args:
        pred_box: shape (4, ):  y_min, x_min, y_max, x_max - predicted box
        gt_boxes: shape (n, 4): y_min, x_min, y_max, x_max - ground truth
        h: image height
        w: image width
    """

    # Transpose the coordinates from y_min, x_min, y_max, x_max
    # In absolute coordinates to x_min, y_min, x_max, y_max
    # in pixel coordinates
    def _swap(box):
        return tf.stack([box[1] * w, box[0] * h, box[3] * w, box[2] * h])

    pred_box = _swap(pred_box)
    gt_box = _swap(gt_box)

    box_area = (pred_box[2] - pred_box[0]) * (pred_box[3] - pred_box[1])
    area = (gt_box[2] - gt_box[0]) * (gt_box[3] - gt_box[1])
    xx1 = tf.maximum(pred_box[0], gt_box[0])
    yy1 = tf.maximum(pred_box[1], gt_box[1])
    xx2 = tf.minimum(pred_box[2], gt_box[2])
    yy2 = tf.minimum(pred_box[3], gt_box[3])

    # compute the width and height of the bounding box
    w = tf.maximum(0, xx2 - xx1)
    h = tf.maximum(0, yy2 - yy1)

    inter = w * h
    return inter / (box_area + area - inter)
````

e la soglia di detection pari di IoU >= 0.75, misurare la `tf.keras.metric.Precision` durante il training ed in validation.

Interrrompere il train quando il modello ha raggiunto un valore circa costante (+/- un valore epsilon arbitrario) sia per il valore di train IoU che di train accuracy.